# VaR - GARCH

## 1. Value-at-Risk

There are several methods to evaluate risk for an individual stock or a portfolio, such as variance, standard deviation of returns, et al. But, those measures do not consider a probability distribution. However, many risk managers prefer a simple measure called Value at Risk (VaR). VaR is one of the most important metrics that is used to measure the risk associated with a financial position or a portfolio of financial instruments and can be defined as the maximum loss with a confidence level over a predetermined period.

Let’s say that the 1-day 95% VaR of our portfolio is $100. This means that 95% of the time, it is expected that - under normal market conditions - we will not lose more than $100 by holding our portfolio over one day.

Three approaches that are commonly used in the industry are:
* Parametric
* Historical
* Monte Carlo

### Import Libraries

We’ll import the required libraries that we’ll use in this example.

In [1]:
# Data manipulation
import warnings
import pandas as pd
import numpy as np
from pprint import pprint
from collections import OrderedDict
from numpy.linalg import multi_dot

from scipy import stats
from tabulate import tabulate

# Import plotly express
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)


### Retrieve Data

We will use the Indian stocks as before to build for calculation of VaR.

In [2]:
# Read from file
df = pd.read_csv('../03_portfolio_optimization/india_stocks.csv',
                 index_col=0, parse_dates=True)
# Display dataframe
df.head()


,ASIANPAINT.NS,HDFCBANK.NS,ITC.NS,RELIANCE.NS,TCS.NS
Date,,,,,
2015-01-01,704.9195,446.3565,191.1795,417.9889,1065.1384
2015-01-02,729.2902,452.5686,191.7784,416.8826,1079.3241
2015-01-05,729.2433,448.7476,192.5334,412.3162,1062.9211
2015-01-06,711.8354,441.7619,187.5866,393.6035,1023.7347
2015-01-07,726.1379,443.0512,184.0979,402.1713,1011.6423


In [3]:
# Calculate daily returns
returns = df.pct_change().dropna()
returns.head()


,ASIANPAINT.NS,HDFCBANK.NS,ITC.NS,RELIANCE.NS,TCS.NS
Date,,,,,
2015-01-02,3.4572e-02,0.0139,0.0031,-0.0026,0.0133
2015-01-05,-6.4275e-05,-0.0084,0.0039,-0.0110,-0.0152
2015-01-06,-2.3871e-02,-0.0156,-0.0257,-0.0454,-0.0369
2015-01-07,2.0092e-02,0.0029,-0.0186,0.0218,-0.0118
2015-01-08,6.3043e-02,0.0210,0.0250,-0.0143,0.0108


In [4]:
# Plot histogram
px.histogram(returns,
             histnorm='probability density',
             title='Histogram of Returns',
             barmode='relative')


### 1.1 Parametric VaR

The Variance-covariance is a parametric method which assumes (almost always) that the returns are normally distributed. In this method, we first calculate the mean and standard deviation of the returns to derive the risk metric. Based on the assumption of normality, we can generalise, $\text{VaR} = \text{position} \times (\mu - z \times \sigma)$

|Confidence Level   |Value At Risk|
|-----|------|
|90%|$\mu - 1.29 \times \sigma$|
|95%|$\mu - 1.64 \times \sigma$|
|99%|$\mu - 2.33 \times \sigma$|

where, $\mu$ is the return, $\sigma$ is the volatility and $z$ is the number of standard deviation from the mean.

In [6]:
# Stock returns
stockreturn = returns['HDFCBANK.NS']
# Calculate mean and standard deviation
mean = np.mean(stockreturn)
stdev = np.std(stockreturn)
# Calculate VaR at difference confidence level
VaR_90 = stats.norm.ppf(1-0.90,mean,stdev)
VaR_95 = stats.norm.ppf(1-0.95,mean,stdev)
VaR_99 = stats.norm.ppf(1-0.99,mean,stdev)

In [7]:
# number of stdev from the mean
stats.norm.ppf(0.01)

-2.3263478740408408

In [8]:
# Ouput results in tabular format
table = [['90%', VaR_90],['95%', VaR_95],['99%', VaR_99] ]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(table,headers=header))

Confidence Level      Value At Risk
------------------  ---------------
90%                      -0.0181096
95%                      -0.0234583
99%                      -0.0334916


#### 1.1.1 Normality Test


In the Parametric VaR, we assumed that the returns are normally distributed. However, in the real world, we know that stock / portfolio returns do not necessarily follow a normal distribution. Let’s perform a quick check to determine the normality of the underlying returns and see whether we need to modify our approach in deriving the VaR numbers.

Shapiro The Shapiro-Wilk test is a test of normality and is used to determine whether or not a sample comes from a normal distribution.

In [9]:
# normality test
stats.shapiro(stockreturn)

ShapiroResult(statistic=0.9051820039749146, pvalue=2.305284011916666e-33)

Our null hypothesis is that HDFCBank stock daily returns follows a normal distribution. Since the p-value is less than 0.05, we reject the null hypothesis. We have sufficient evidence to say that the sample data does not come from a normal distribution. This result shouldn’t be surprising as the data comes from an empirical distribution.

Anderson-Darling Alternatively, we can perform an Anderson-Darling Test. It is a goodness of fit test that measures how well the data fit a specified distribution. This test is most commonly used to determine whether or not the data follow a normal distribution.

In [10]:
# normality test
stats.anderson(stockreturn)

AndersonResult(statistic=27.425866045315615, critical_values=array([0.575, 0.655, 0.785, 0.916, 1.09 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.000758040408841441, scale=0.014726218186539864)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

Based on the above result, the null hypothesis is rejected since the test statistic value is much higher than the critical value of 1.09 even at 1% significance level.

### 1.2 Modified VaR

Standard normal distribution have a zero mean, unit variance, zero skewness, and its kurtosis of 3. However, we now know the distribution is not normal and in such scenario, the skewness and excess kurtosis of many stock returns are not zero. As a consequence, the modified VaR was developed to utilize those four moments instead of the first two moments.
$$
m \text{VaR} = \text{position} \times (\mu - t \times \sigma)
$$
where,
$$
t = z + \dfrac{1}{6}(z^2 - 1)s + \dfrac{1}{24} (z^3 - 3z)k - \dfrac{1}{36}(2z^3 - 5z)s^2
$$

$\mu$ is the return, $\sigma$ is the volatility, $s$ is the skewness, $k$ is the kurtosis and $z$ is the absolute number of standard deviation from the mean.


In [12]:
# First four moments
dist = OrderedDict({
    'Mean': np.mean(returns['HDFCBANK.NS']),
    'Variance': np.std(returns['HDFCBANK.NS']),
    'Skew': stats.skew(returns['HDFCBANK.NS']),
    'Kurtosis': stats.kurtosis(returns['HDFCBANK.NS'])
    })
pprint(dist)

OrderedDict([('Mean', 0.000758040408841441),
             ('Variance', 0.014722489557998887),
             ('Skew', 0.07119829662074695),
             ('Kurtosis', 10.459975492931424)])


In [13]:
# Specify params
z = abs(stats.norm.ppf(0.01))
s = stats.skew(stockreturn)
k = stats.kurtosis(stockreturn)
t = z+1/6*(z**2-1)*s+1/24*(z**3-3*z)*k-1/36*(2*z**3-5*z)*s**2
# Calculate VaR at difference confidence level
mVaR_99 = (mean-t*stdev)
mVaR_99

-0.07023685594090116

### 1.3 Historical VaR

Asset returns do not necessarily follow a normal distribution. An alternative is to use sorted returns to evaluate a VaR. This method uses historical data where returns are sorted in ascending order to calculate maximum possible loss for a given confidence level.

In [14]:
# Use quantile function for Historical VaR
hVaR_90 = returns['HDFCBANK.NS'].quantile(0.10)
hVaR_95 = returns['HDFCBANK.NS'].quantile(0.05)
hVaR_99 = returns['HDFCBANK.NS'].quantile(0.01)

In [15]:
# Ouput results in tabular format
htable = [['90%', hVaR_90],['95%', hVaR_95],['99%', hVaR_99]]
print(tabulate(htable,headers=header))

Confidence Level      Value At Risk
------------------  ---------------
90%                      -0.0141971
95%                      -0.0199752
99%                      -0.0364953


### 1.4 MonteCarlo VaR

The Monte Carlo simulation approach has a number of similarities to historical simulation. It allows us to use actual historical distributions rather than having to assume normal returns. As returns are assumed to follow a normal distribution, we could generate n simulated returns with the same mean and standard deviation (derived from the daily returns) and then sorted in ascending order to calculate maximum possible loss for a given confidence level.

In [16]:
# Set seed for reproducibility
np.random.seed(42)
# Number of simulations
n_sims = 5000
# Simulate returns and sort
sim_returns = np.random.normal(mean, stdev, n_sims)
# Use percentile function for MCVaR
MCVaR_90 = np.percentile(sim_returns,10)
MCVaR_95 = np.percentile(sim_returns, 5)
MCVaR_99 = np.percentile(sim_returns,1)

In [17]:
# Ouput results in tabular format
mctable = [['90%', MCVaR_90],['95%', MCVaR_95],['99%', MCVaR_99]]
print(tabulate(mctable,headers=header))

Confidence Level      Value At Risk
------------------  ---------------
90%                      -0.0179306
95%                      -0.0229988
99%                      -0.0340315


### 1.5 Scaling VaR

Now, let’s calculate VaR over a 5-day period. To scale it, multiply by square root of time.
$$
\text{VaR} = \text{position} \times (\mu - z \times \sigma) \times \sqrt{T}
$$

where, $T$ is the horizon or forecast period.

In [18]:
# VaR Scaling
forecast_days = 5
f_VaR_90 = VaR_90*np.sqrt(forecast_days)
f_VaR_95 = VaR_95*np.sqrt(forecast_days)
f_VaR_99 = VaR_99*np.sqrt(forecast_days)

In [19]:
# Ouput results in tabular format
ftable = [['90%', f_VaR_90],['95%', f_VaR_95],['99%', f_VaR_99] ]
fheader = ['Confidence Level', '5-Day Forecast Value At Risk']
print(tabulate(ftable,headers=fheader))

Confidence Level      5-Day Forecast Value At Risk
------------------  ------------------------------
90%                                     -0.0404943
95%                                     -0.0524544
99%                                     -0.0748895


In [20]:
# Plot Scaled VaR
sVaR = pd.DataFrame([-100*VaR_99*np.sqrt(x) for x in range(100)], columns=['ScaledVaR'])
px.scatter(sVaR, sVaR.index,'ScaledVaR',title='Scaled VaR', labels={'index': 'Horizon'})

### 1.6 Expected Short Fall

VaR is a reasonable measure of risk if assumption of normality holds. Else, we might underestimate the risk if we observe a fat tail or overestimate the risk if tail is thinner. Expected shortfall or Conditional Value at Risk - CVaR - is an estimate of expected shortfall sustained in the worst 1 - x% of scenarios. It is defined as the average loss based on the returns that are lower than the VaR threshold. Assume that we have n return observations, then the expected shortfall is

$$
\text{CVaR} = \frac{1}{n} \sum\limits_{i=1}^{n} R_i [R \leq h \text{VaR}_{cl}]
$$

where, R is returns, hVaR is historical VaR and cl is the confidence level.

In [21]:
# Calculate CVar
CVaR_90 = returns['HDFCBANK.NS'][returns['HDFCBANK.NS']<=hVaR_90].mean()
CVaR_95 = returns['HDFCBANK.NS'][returns['HDFCBANK.NS']<=hVaR_95].mean()
CVaR_99 = returns['HDFCBANK.NS'][returns['HDFCBANK.NS']<=hVaR_99].mean()

In [22]:
# Ouput results in tabular format
ctable = [['90%', CVaR_90],['95%', CVaR_95],['99%', CVaR_99] ]
cheader = ['Confidence Level', 'Conditional Value At Risk']
print(tabulate(ctable,headers=cheader))

Confidence Level      Conditional Value At Risk
------------------  ---------------------------
90%                                  -0.0242577
95%                                  -0.0319729
99%                                  -0.0560803


### 1.7 Portfolio VaR

If we know the returns and volatilities of all the assets in the portfolio, we can derive portfolio VaR. We will now derive VaR of minimum variance portfolio consisting of Indian stocks.

In [23]:
# Weights from Minimum Variance Portfolio
wts = np.array([1.928e-01, 2.367e-01, 2.099e-01, 7.286e-02, 2.878e-01])
# Portfolio mean returns and volatility
port_mean = wts.T @ returns.mean()
port_stdev = np.sqrt(multi_dot([wts.T, returns.cov(), wts]))
pVaR = stats.norm.ppf(1-0.99, port_mean, port_stdev)
print(f"Mean: {port_mean}, Stdev: {port_stdev}, pVaR: {pVaR}")

Mean: 0.0007072635611216401, Stdev: 0.010385536949914594, pVaR: -0.023453108243084775


## 2. GARCH

Asset price volatility is central to derivatives pricing. It is defined as measure of price variability over certain period of time. In essence, it describes standard deviation of returns. There are different types of volatility: Historical, Implied, Forward. In most cases, we assume volatility to be constant, which is clearly not true and numerous studies have been dedicated to estimate this variable, both in academia and industry.

### Volatility

Volatility estimation by statistical means assume equal weights to all returns measured over the period. We know that over 1-day, the mean return is small as compared to standard deviation. If we consider a simple m-period moving average, where $\sigma_n$ is the volatility of return on day n, then with $\bar u \approx 0$, we have

In [ ]:
## 